# Testando site SATVIEW x sgp4 + py3d

**Links**

- http://www.satview.org/
- https://www.n2yo.com/satellite/?s=25544

# Unidades

Ângulos: em graus

Distância: metros (m)

Tempo: UTC

In [1]:
%load_ext autoreload

%autoreload 2

In [9]:
from datetime import datetime, timezone

## Constants

In [3]:
OBS_LAT = -15.989620
OBS_LON = -48.044411
OBS_ALT = 500 # meters

ISS_TLE = ('1 25544U 98067A   18264.87214538 -.00000479 +00000-0 +00000-0 0 9990',
           '2 25544 051.6411 256.8919 0004651 185.2142 246.8508 15.53744581133552')

# SATVIEW x py3d

In [4]:
from sgp4.earth_gravity import wgs84
from sgp4.io import twoline2rv
import pymap3d as pm


def propagate_satellite(satellite, at):
    date_at = at.year, at.month, at.day, at.hour, at.minute, at.second + (at.microsecond / 1000000.0)
    return satellite.propagate(*date_at)

satellite = twoline2rv(ISS_TLE[0], ISS_TLE[1], wgs84)
utc_satview = datetime(year=2018, month=9, day=22, hour=20, minute=37, second=30, tzinfo=timezone.utc)

iss_position, v = propagate_satellite(satellite, utc_satview)
iss_position = (iss_position[0] * 1000.0, iss_position[1] * 1000.0, iss_position[2] * 1000.0)

print('iss_position = {}'.format(iss_position))

iss_position = (768350.0713588104, 6538760.196343955, -1632694.711713458)


In [5]:
import numpy as np


iss_norm = np.linalg.norm(iss_position)
print('iss_norm = {}'.format(iss_norm / 1000.0))
print('iss_norm - 400 = {}'.format((iss_norm / 1000.0) - 400))

iss_norm = 6783.173207216974
iss_norm - 400 = 6383.173207216974


In [6]:
eci_iss = iss_position

iss_aer = pm.eci2aer(eci_iss, OBS_LAT, OBS_LON, OBS_ALT, utc_satview)

print('iss_aer = {}'.format(iss_aer))
iss_az = iss_aer[0][0]
iss_el = iss_aer[1][0]

iss_aer = (array([180.60183682]), array([-74.51896759]), array([12717620.45332964]))


In [7]:
pm.eci2aer??

In [8]:
print('Py3d')
print('azimuth = {}'.format(iss_az))
print('elevation = {}'.format(iss_el))
print('local_time = {}'.format(utc_satview))

Py3d
azimuth = 180.60183682285862
elevation = -74.51896759111999
local_time = 2018-09-22 20:37:30+00:00


# Conclusão 